## DEMO 6

In [ ]:
# cargo el dataset

import pandas as pd
df = pd.read_csv("T_C02-1C2025T-helados2.csv", sep=";")
df.head()

In [ ]:
# separo X (datos de entrada) e y (resultado conocido)

y = df['KG']
X = df.drop(columns=['KG'])

In [ ]:
# scalo los datos ... (X-u)/d
from sklearn.preprocessing import StandardScaler
e = StandardScaler()
X = e.fit_transform(X)      

In [ ]:
# divido el dataset entre entrenamiento (80%) y test (20%)

from sklearn.model_selection import train_test_split

print('tamaño X: ', X.shape)
print('tamaño y: ', y.shape)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2, random_state=9)

print('train X', X_train.shape, 'train y', y_train.shape)
print('test X', X_test.shape, 'test y', y_test.shape)

In [ ]:
import torch    # importo la biblioteca de IA que voy a usar, en este caso pyTorch 

device = 'cpu' # 'cpu' 'cuda' ¿qué dispositivo va a procesar los datos?

X_train_t = torch.tensor(X_train).float().to(device)
X_test_t = torch.tensor(X_test).float().to(device)
y_train_t = torch.tensor(y_train.values).float().to(device)
y_test_t = torch.tensor(y_test.values).float().to(device)
y_test_t = y_test_t[:, None]
y_train_t = y_train_t[:, None]

In [ ]:
# definimos e instanciamos el modelo de la red neuronal

model = torch.nn.Sequential(
    torch.nn.Linear(2, 6),
    torch.nn.ReLU(),
    torch.nn.Linear(6, 4),
    torch.nn.ReLU(),
    torch.nn.Linear(4, 1),
)

print(model)    # imprimo el modelo de red neuronal

In [ ]:
# defino los hiperparámetros (parametros de entrenamiento) del modelo
lr = 0.001
epochs = 50000
e_print = 100
loss_fn = torch.nn.MSELoss()
optimizer = torch.optim.Adam(params=model.parameters(), lr=lr)

In [ ]:
# entreno el modelo

for epoch in range(1, epochs+1):
    y_pred = model(X_train_t)
    loss = loss_fn(y_pred, y_train_t)
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()
    
    if epoch % e_print == 0:
        print(f"epoch {epoch} \t Loss: {round(loss.item(),4)}")

In [ ]:
# pruebo el modelo:

X = [[20.0, 0]] # el primer valos es el de temperatura, el segundo si es fin de semana o no

Xp = e.transform(X) # escalo mis datos de entrada
print(Xp)

pred = model(torch.tensor(Xp).float())

print(pred) # imprimo la predicción